This week's task is to try to run some taggers on the Audible data given to us by Dan. We have two sets of files. The first is an unlabeled dataset that contains approximately 25% of the text from 1000 books. The second dataset consists of 10 books from the first dataset in XML files that are labeled with parts of speech, word chunks (i.e. verb phrases, noun phrases), lemmatized, and the basic entities (People, Organizations, and Locations). Let's do some exploration of these files.

First, let's find out how many individual words are given entity tags, and what type of tags they are:

In [1]:
import lxml.etree as etree
from bs4 import BeautifulSoup
import glob
from __future__ import division

files = glob.glob('data/audible/processedText/*.xml')
entities = {'P': 0, 'L':0, 'O':0, 'D':0}
total_tokens = 0
for f in files:
    xml_data = open(f).read()
    soup = BeautifulSoup(xml_data, 'lxml')
    for i in soup.find_all(['w', 'c']):
        for item in i:
            total_tokens += 1
            if 'ner' in i.attrs:
                entities[i.attrs['ner'][0]] += 1

In [2]:
for i in entities.keys():
    print('There are ' + str(entities[i]) + ' instances of ' + str(i))
print('***** ' + str(total_tokens) + ' tokens ******')

There are 9057 instances of P
There are 1407 instances of L
There are 391 instances of O
There are 0 instances of D
***** 326316 tokens ******


In the labeled data, we see that there are just over 326k individual tokens, and nearly 11k of them are labeled as entities. The majority of these entities (~9k) are people, with a few location tags, and a small number of Organizations. Dates are not labeled in these data, so we'll exclude them from our experimentation.

One thing to note is that this method of counting double-counts some instances. For instance:

In [3]:
files = glob.glob('data/audible/processedText/*.xml')
total_tokens = 0
unique_entities = {}
for f in files:
    xml_data = open(f).read()
    soup = BeautifulSoup(xml_data, 'lxml')
    for i in soup.find_all(lambda tag: 'ner' in tag.attrs):
        if i['ner']+'_'+f[-14:-4] in unique_entities.keys():
            unique_entities[i['ner']+'_'+f[-14:-4]] += 1
        else:
            unique_entities[i['ner']+'_'+f[-14:-4]] = 1
        

In [4]:
import operator
longest_entity = max(unique_entities.iteritems(), key=operator.itemgetter(1))
ent_id, file_id = longest_entity[0].split('_')
xml_data = open('data/audible/processedText/'+file_id+'.xml').read()
soup = BeautifulSoup(xml_data, 'lxml')
entity = soup.findAll(ner=ent_id)
print('The entity that takes up the largest number of strings is:')
print(' '.join(i.text for i in entity))
print('\n')
print('*****************')
print('The entire sentence that contains this entity is:')
print(' '.join(entity[0].parent.parent.text.split()))

The entity that takes up the largest number of strings is:
Harmonic Field of Glass Bells and Green Gig


*****************
The entire sentence that contains this entity is:
We might add here that later on the constructors had an article published in a prominent scientific journal under the title of ' Recursive β – Metafunctions in the Special Case of a Bogus Polypolice Transmogrification Conversion on an Oscillating Harmonic Field of Glass Bells and Green Gig , Kerosene Lamp on the Left to Divert Attention , Solved by Beastly Incarceration – Concatenation ' , which was subsequently exploited by the tabloids as ' The Police State Rears Its Ugly Head ' .


In the former counting method, this entity would have been counted as 8 distinct entities, whereas it's actually meant to be one entity. Interestingly, this example is incorrectly labeled. This entity is labeled as an organization, but it's actually part of a title (and would make a great band name!). Regardless, this highlights the importance of why using a standard scoring approach wont work so well.

# Spacy Entities

Below, I explore how Spacy handles entities in this text. The standard way of scoring this type of data is the one used for the CONLL shared tasks. The perl CONLL scoring script is available [here](http://www.cnts.ua.ac.be/conll2002/ner/bin/conlleval.txt), and I use this as my scorer. One of the difficulties here is that the spacy tokenizer works a bit differently than the tokenizer used on the Audible data. Below, I print out a sample of one of the documents with the ner tags. The printout is in the following format:


|spacy_text   |spacy_tag   |******   |audible_text   |audible_tag   |
|---|---|--:|---|---|
|disemboweled   |O-   |******   |disemboweled   |O   |
|,   |O-   |******   |,   |O   |
|buried   |O-   |******   |buried   |O   |


In [5]:
import spacy
nlp = spacy.load('en')

In [6]:
words = soup.find_all(['w', 'c'])
doc = nlp(' '.join(w.text for w in words))
start = 175
subdoc = doc[start:220]
for i, w in enumerate(subdoc):
    if 'ner' in words[i+start].attrs:
        print w.text + '\t' + w.ent_iob_ + '-' + w.ent_type_ + '\t******  ' + words[i+start].text + '\t' + words[i+start].attrs['ner']
    else:
        print w.text + '\t' + w.ent_iob_ + '-' + w.ent_type_ + '\t******  ' + words[i+start].text + '\t' + 'O'

disemboweled	O-	******  disemboweled	O
,	O-	******  ,	O
buried	O-	******  buried	O
alive	O-	******  alive	O
,	O-	******  ,	O
crucified	O-	******  crucified	O
and	O-	******  and	O
burnt	O-	******  burnt	O
at	O-	******  at	O
the	O-	******  the	O
stake	O-	******  stake	O
,	O-	******  ,	O
after	O-	******  after	O
which	O-	******  which	O
your	O-	******  your	O
ashes	O-	******  ashes	O
shall	O-	******  shall	O
be	O-	******  be	O
sent	O-	******  sent	O
into	O-	******  into	O
orbit	O-	******  orbit	O
as	O-	******  as	O
a	O-	******  a	O
warning	O-	******  warning	O
and	O-	******  and	O
perpetual	O-	******  perpetual	O
reminder	O-	******  reminder	O
to	O-	******  to	O
all	O-	******  all	O
would	O-	******  would-be	O
-	O-	******  regicides	O
be	O-	******  ,	O
regicides	O-	******  amen	O
,	O-	******  .	O
amen	O-	******  '	O
.	O-	******  '	O
'	O-	******  Ca	O
'	O-	******  n't	O
Ca	O-	******  you	O
n't	O-	******  wait	O
you	O-	******  a	O
wait	O-	******  bit	O
a	O-	******  ?	O
bit	O-	******  '	O
?	

One can see that the compound word would-be (about halfway down) is treated differently by the two systems. This makes lining up the annotations a bit of a challenge. Especially since it may mean that Spacy will provide multiple tags to the same word which is given only one tag from Audible. To counteract this, I'll group together the words that are split apart, and give them an entity tag if any of the tokens contains one. Fortunately, there were no cases where these split words contained two different tags.

I've collected some utility functions (e.g. the one that condenses tags as described above) and placed them in a separate file, `audible.py` found [here](https://github.com/riddlet/Audible/blob/master/audible.py)

In [7]:
# ugh, this is ugly.
from audible import condense_tokens

spacy_text = []
spacy_tag = []
audible_text = []
audible_tag = []
for f in files:
    xml_data = open(f).read()
    soup = BeautifulSoup(xml_data, 'lxml')
    words = soup.find_all(['w', 'c'])
    doc = nlp(' '.join(w.text for w in words))
    offset = [0, 0] #spacy, audible
    for i, w in enumerate(doc):
        if i+offset[1] == len(words):
            break
        spacy_tokens = nlp(words[i+offset[1]].text) #check to see if the tokenizations match

        if len(spacy_tokens)==1: #if they match
            if 'ner' in words[i+offset[1]].attrs:
                spacy_text.append(doc[i+offset[0]].text)
                spacy_tag.append(doc[i+offset[0]].ent_iob_ + '-' + doc[i+offset[0]].ent_type_)
                audible_text.append(words[i+offset[1]].text)
                audible_tag.append(words[i+offset[1]].attrs['ner'])
            else:
                spacy_text.append(doc[i+offset[0]].text)
                spacy_tag.append(doc[i+offset[0]].ent_iob_ + '-' + doc[i+offset[0]].ent_type_)
                audible_text.append(words[i+offset[1]].text)
                audible_tag.append('O')
        if len(spacy_tokens)>1: #if spacy splits it up into more tokens
            stext, stag, atext, atag, offset = condense_tokens(doc, words, i, offset, nlp)
            spacy_text.append(stext)
            spacy_tag.append(stag)
            audible_text.append(atext)
            audible_tag.append(atag)

In [8]:
import pandas as pd
df = pd.DataFrame({'spacy_text':spacy_text,
             'spacy_tag':spacy_tag,
             'audible_text':audible_text,
             'audible_tag':audible_tag})

## Standardizing Tags
Spacy's entity recognizer includes many other categories that are not listed in the Audible data. The next bits of code removes those entities from the tags provided by Spacy and then makes the remaining ones in a standard format that is similar to that expected by the CONLL script. I then standardize the tags from the Audible data.

In [9]:
import re
from audible import clean_spacy_tags

df['cleaned_spacy_tags'] = df.spacy_tag.apply(clean_spacy_tags)

df.cleaned_spacy_tags.value_counts()

O-     315119
B-P      7195
B-L      1341
I-P       977
B-O       904
I-O       592
I-L       188
Name: cleaned_spacy_tags, dtype: int64

The above provides some initial indication that Spacy is a reasonable approximation to the tags in the Audible data. We can tell this because People tags make up a larger share of the tags than do Locations or Organizations. Though we also see that there appear to be a larger number of organizations (904) here than in the audible data (391).

I now convert the audible tags into the same format.

In [10]:
from audible import clean_audible_tags

df['aud_clean'] = df.audible_tag.apply(clean_audible_tags)
df['counter'] = df.groupby('audible_tag').cumcount()

df['iob_sys'] = 'O'

df['iob_sys'][df.counter==0] = 'B'
df['iob_sys'][df.counter>0] = 'I'
df['aud_clean'][df.aud_clean!='O-'] = df['iob_sys']+df['aud_clean']
df.aud_clean.value_counts()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


O-     315461
I-P      5857
B-P      3200
B-L       743
I-L       664
I-O       235
B-O       156
Name: aud_clean, dtype: int64

## Spacy Scoring
Below, I write a file for the conll script. I think that evaluation here consists of entire chunks, rather than individualized tokens. As such, if System one identified "Susan Brown" as one entity chunk, and system two only identified "Susan" as the chunk, then this would be incorrect.

The performance below is treating the audible data as the 'gold standard'. We can see that there's generally good agreement, though it seems as though the audible and spacy systems frequently disagree on organizational tags, and Location tags have pretty middling performance.

In [256]:
outdat = df[['audible_text', 'aud_clean', 'cleaned_spacy_tags']]
outdat['audible_text'] = outdat.audible_text.apply(lambda x: x.replace(' ', ''))
outdat.to_csv('temp', sep=' ', encoding='utf-8', index=False, header=False)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


processed 326316 tokens with 9108 phrases; found: 9488 phrases; correct: 6682.

`accuracy:  97.33%; precision:  70.43%; recall:  73.36%; FB1:  71.86`

                L: precision:  48.88%; recall:  62.96%; FB1:  55.03  1342
                
                O: precision:   5.53%; recall:  25.00%; FB1:   9.06  940
                
                P: precision:  82.90%; recall:  76.02%; FB1:  79.31  7206

We can see above that performance is a reasonable approximation to the Audible tags. Identifying Persons is most consistent, with Locations producing more middling results, and Organizations matching rather poorly.

# CoreNLP system

Dan's initial impression was that the Kindle system audible uses to tag their books is based on Stanford's CoreNLP system, or at least was very similar. Here, I'm using the NLTK wrapper to stanford's tagger. As in the spacy case, I've also built in some tools to handle instances in which the tokenization between the two systems does not match. This may result in some biases to the evaluation metrics. I haven't done a systematic exploration of the direction of this bias, but I can't think of a reason why this would bias the evaluation upwards. And I suspect that any bias introduced is small (less than 1%).

In [11]:
from nltk.tag.stanford import NERTagger
st = NERTagger('../stanford-ner-2017-06-09/classifiers/english.all.3class.distsim.crf.ser.gz',
               '../stanford-ner-2017-06-09/stanford-ner-3.8.0.jar')

In [12]:
from audible import condense_stanford_tokens

**Note** The cell below takes a long time to run (~24 hours on my 2013-era MacBook Air). This could be reduced through parallelization, or by running the command line tool with many of the extra features turned off. I'm not sure whether this wrapper exposes arguments to switch off these features.

In [506]:
stanford_text = []
stanford_tag = []
audible_text = []
audible_tag = []
for f in files:
    print f
    xml_data = open(f).read()
    soup = BeautifulSoup(xml_data, 'lxml')
    sentences = soup.find_all('s')
    for j, s in enumerate(sentences):
        if j % 100 == 0:
            print '%s of document complete' % (j/len(sentences))
        words = s.find_all(['w', 'c'])
        doc = ' '.join(w.text.encode('utf8') for w in words)
        try:
            stanford_words = st.tag([doc])
        except:
            stanford_words = st.tag([doc.decode('ascii', 'ignore')])
        if len(stanford_words) == len(words):
            for i, w in enumerate(words):
                if 'ner' in w.attrs:
                    stanford_text.append(stanford_words[i][0])
                    stanford_tag.append(stanford_words[i][1])
                    audible_text.append(w.text)
                    audible_tag.append(w.attrs['ner'])
                else:
                    stanford_text.append(stanford_words[i][0])
                    stanford_tag.append(stanford_words[i][1])
                    audible_text.append(w.text)
                    audible_tag.append('O-')
        else:
            stanford_text, stanford_tag, audible_text, audible_tag = condense_stanford_tokens(s, stanford_text, stanford_tag, audible_text, audible_tag, st)
            

data/audible/processedText/B000FCKA6C.xml
0.0 of document complete
0.0473260766682 of document complete
0.0946521533365 of document complete
0.141978230005 of document complete
0.189304306673 of document complete
0.236630383341 of document complete
0.283956460009 of document complete
0.331282536678 of document complete
0.378608613346 of document complete
0.425934690014 of document complete
0.473260766682 of document complete
0.520586843351 of document complete
0.567912920019 of document complete
0.615238996687 of document complete
0.662565073355 of document complete
0.709891150024 of document complete
0.757217226692 of document complete
0.80454330336 of document complete
0.851869380028 of document complete
0.899195456697 of document complete
0.946521533365 of document complete
0.993847610033 of document complete
data/audible/processedText/B003ZSHUK2.xml
0.0 of document complete
0.043956043956 of document complete
0.0879120879121 of document complete
0.131868131868 of document complete


In [511]:
df_corenlp = pd.DataFrame({'stanford_text':stanford_text,
             'stanford_tag':stanford_tag,
             'audible_text':audible_text,
             'audible_tag':audible_tag})

## Standardizing Tags

As before, we need to standardize the tags to make the suitable for comparing with each other and for running through the conll scoring script.

In [538]:
from audible import clean_stanford_tags

df_corenlp['cleaned_stanford_tags'] = df_corenlp.stanford_tag.apply(clean_stanford_tags)

df_corenlp['prev_tag'] = df_corenlp.cleaned_stanford_tags.shift(1)

df_corenlp['iob'] = 'O'
df_corenlp['iob'][(df_corenlp.cleaned_stanford_tags!='O-') & 
                  (df_corenlp.cleaned_stanford_tags != df_corenlp.prev_tag)] = 'B'
df_corenlp['iob'][(df_corenlp.cleaned_stanford_tags!='O-') & 
                 (df_corenlp.cleaned_stanford_tags == df_corenlp.prev_tag)] = 'I'


df_corenlp['cleaned_stanford_tags'][df_corenlp.cleaned_stanford_tags!='O-'] = df_corenlp['iob']+df_corenlp['cleaned_stanford_tags']
df_corenlp.cleaned_stanford_tags.value_counts()

#df_corenlp.cleaned_stanford_tags.value_counts()
#df[df.cleaned_spacy_tags=='O-P']

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


O-     315619
B-P      7735
B-L      1126
I-P       920
B-O       373
I-L       317
I-O       226
Name: cleaned_stanford_tags, dtype: int64

In [541]:

df_corenlp['aud_clean'] = df_corenlp.audible_tag.apply(clean_audible_tags)
df_corenlp['counter'] = df_corenlp.groupby('audible_tag').cumcount()

df_corenlp['iob_sys'] = 'O'

df_corenlp['iob_sys'][df_corenlp.counter==0] = 'B'
df_corenlp['iob_sys'][df_corenlp.counter>0] = 'I'
df_corenlp['aud_clean'][df_corenlp.aud_clean!='O-'] = df_corenlp['iob_sys']+df_corenlp['aud_clean']
df_corenlp.aud_clean.value_counts()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


O-     315461
I-P      5857
B-P      3200
B-L       743
I-L       664
I-O       235
B-O       156
Name: aud_clean, dtype: int64

In [543]:
outdat = df_corenlp[['audible_text', 'aud_clean', 'cleaned_stanford_tags']]
outdat['audible_text'] = outdat.audible_text.apply(lambda x: x.replace(' ', ''))
outdat.to_csv('temp_stanford', sep=' ', encoding='utf-8', index=False, header=False)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


`processed 326316 tokens with 9108 phrases; found: 9234 phrases; correct: 7674.`

`accuracy:  97.71%; precision:  83.11%; recall:  84.26%; FB1:  83.68
                L: precision:  68.21%; recall:  73.70%; FB1:  70.85  1126
                O: precision:  19.30%; recall:  34.62%; FB1:  24.78  373
                P: precision:  88.35%; recall:  86.97%; FB1:  87.65  7735`

Above, we see that, just as Dan had suspected, the CoreNLP system is a closer approximation to whatever Kindle is using than is the Spacy system. Also interesting is that the performance across the three classes shows the same pattern as the Spacy system. That is, there's relatively good agreement for Persons, and relatively poor agreement on Organizations, with Locations placed in the middle.

For next time, the clearest course of action is perform some error analysis. I'll examine places where these systems disagree to see if there's some obvious reason for these disparities.